In [1]:
%matplotlib inline

from scipy.odr import *
from scipy.stats import *
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import ast
from multiprocessing import Pool, cpu_count

import ipywidgets as widgets

import scipy

from IPython import display
from matplotlib.patches import Rectangle

from sklearn.metrics import mean_squared_error
import json

import scipy.stats as st
from sklearn.metrics import r2_score


from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

import copy

from sklearn.model_selection import LeaveOneOut, LeavePOut

from multiprocessing import Pool
import cv2

In [2]:
df_filtered = pd.read_pickle("PklData/dfFiltered_LSTM.pkl")

In [3]:
df_filtered.head()

,userID,Timestamp,Current_Task,Task_amount,TaskID,VersionID,RepetitionID,Actual_Data,Is_Pause,Image
8185,17,1547138938136,2,680,16,2,0,True,False,"[0, 0, 192, 0, 0, 0, 160, 0, 0, 0, 1, 0, 1, 0,..."
8186,17,1547138938173,2,680,16,2,0,True,False,"[0, 0, 192, 0, 0, 0, 160, 0, 0, 0, 1, 0, 1, 0,..."
8187,17,1547138938217,2,680,16,2,0,True,False,"[0, 0, 192, 0, 0, 0, 160, 0, 0, 0, 1, 0, 1, 0,..."
8188,17,1547138938253,2,680,16,2,0,True,False,"[0, 0, 192, 0, 0, 0, 160, 0, 0, 0, 1, 0, 1, 0,..."
8189,17,1547138938311,2,680,16,2,0,True,False,"[0, 0, 192, 0, 0, 0, 160, 0, 0, 0, 1, 0, 1, 0,..."


In [4]:
df_filtered.Image = df_filtered.Image.apply(lambda x: x.reshape(27, 15))
df_filtered.Image = df_filtered.Image.apply(lambda x: x.clip(min=0, max=255))
df_filtered.Image = df_filtered.Image.apply(lambda x: x.astype(np.uint8))
df_filtered["ImageSum"] = df_filtered.Image.apply(lambda x: np.sum(x))

In [5]:
#LSTMs new Blob detection
def detect_blobs(image):
    #image = image.reshape(27, 15)
    large = np.ones((29,17), dtype=np.uint8)
    large[1:28,1:16] = image
    temp, thresh = cv2.threshold(cv2.bitwise_not(large), 200, 255, cv2.THRESH_BINARY)
    im2, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [a for a in contours if cv2.contourArea(a) > 5 and cv2.contourArea(a) < 255]
    lstBlob  = []
    lstMin = []
    lstMax = []
    count = 0
    return len(contours) > 0

In [6]:
%%time
pool = Pool(cpu_count() - 2)
temp_blobs = pool.map(detect_blobs, df_filtered.Image)
pool.close()

CPU times: user 1.64 s, sys: 531 ms, total: 2.17 s
Wall time: 2.4 s


In [7]:
df_filtered["ContainsBlobs"] = temp_blobs

In [8]:
#Label if knuckle or finger
def f(row):
    if row['TaskID'] < 17:
        #val = "Knuckle"
        val = 0
    elif row['TaskID'] >= 17:
        #val = "Finger"
        val = 1
    return val
df_filtered['InputMethod'] = df_filtered.apply(f, axis=1)

In [9]:
# Problem: some timestamps are strings (XXXXE+XXXX) which is not accurate enough, switching to index instead
"""def cast_to_int(x):
    if type(x) == int:
        return x
    x = str(x).replace(",", ".")
    return int(float(x))

df_filtered.Timestamp = df_filtered.Timestamp.map(cast_to_int)"""
df_filtered.index = range(len(df_filtered))

In [112]:
%%time
UserIDs = []
TaskIDs = []
VersionIDs = []
Blobs = []
for userID in df_filtered.userID.unique():
    print(userID)
    for TaskID in df_filtered[df_filtered.userID == userID].TaskID.unique():
        for VersionID in df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID)].VersionID.unique():
            first_blob = -1
            last_blob = -1
            for index, row in df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID) & (df_filtered.VersionID == VersionID)].iterrows():
                if row.ContainsBlobs:
                    last_blob = index
                    if first_blob == -1:
                        first_blob = index
            if first_blob >= 0 and last_blob >= 0:
                UserIDs.append(userID)
                TaskIDs.append(TaskID)
                VersionIDs.append(VersionID)
                Blobs.append(df_filtered[(df_filtered.userID == userID) & (df_filtered.TaskID == TaskID) & (df_filtered.VersionID == VersionID) & (df_filtered.index >= first_blob) & (df_filtered.index <= last_blob)].Image.tolist())

17
2
12
15
5
1
14
10
13
6
16
3
7
8
9
11
4
CPU times: user 1min 11s, sys: 44.3 ms, total: 1min 11s
Wall time: 1min 11s


In [116]:
np.array(Blobs[0]).shape

(63, 27, 15)

In [117]:
UserIDs = np.array(UserIDs, dtype=np.int64)
TaskIDs = np.array(TaskIDs, dtype=np.int64)
VersionIDs = np.array(VersionIDs, dtype=np.int64)

In [120]:
df_lstm_all = pd.DataFrame()
df_lstm_all["userID"] = UserIDs
df_lstm_all["TaskID"] = TaskIDs
df_lstm_all["VersionID"] = VersionIDs
df_lstm_all["Blobs"] = Blobs
df_lstm_all.Blobs = df_lstm_all.Blobs.map(np.array)
df_lstm_all.head()

,userID,TaskID,VersionID,Blobs
0,17,16,2,"[[[0, 0, 192, 0, 0, 0, 0, 0, 0, 0, 1, 225, 0, ..."
1,17,16,4,"[[[200, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,..."
2,17,16,5,"[[[200, 228, 0, 0, 0, 1, 0, 1, 0, 0, 205, 0, 0..."
3,17,16,6,"[[[201, 0, 0, 1, 165, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,17,16,7,"[[[0, 0, 1, 1, 0, 1, 0, 176, 0, 0, 0, 0, 0, 0,..."


In [126]:
lengths = []
for index, row in df_lstm_all.iterrows():
    lengths.append(row.Blobs.shape[0])
df_lstm_all["BlobCount"] = lengths

In [127]:
print("before: %s" % len(df_lstm_all))
df_lstm = df_lstm_all[(df_lstm_all.BlobCount <= 50) & (df_lstm_all.BlobCount >= 5)]
print("after: %s" % len(df_lstm))
print("ratio: %s" % ((len(df_lstm_all) - len(df_lstm)) / len(df_lstm_all) * 100))

before: 7077
after: 6227
ratio: 12.010739013706374


In [16]:
df_lstm.BlobCount.describe()

count    6227.000000
mean       16.594990
std         8.624577
min         5.000000
25%        10.000000
50%        15.000000
75%        21.000000
max        50.000000
Name: BlobCount, dtype: float64

In [133]:
def lerp(a, b, c=0.5):
    return c * b + (1.0 - c) * a

def normalize_blobs(blobs, new_len=50):
    new_count = new_len - blobs.shape[0]
    if new_count == 0:
        return blobs
    elif new_count > 0:
        temp = np.array([np.zeros((27, 15))] * new_count)
        return np.append(blobs, temp, axis=0)
    else:
        return blobs[0:new_len]

In [135]:
df_lstm_norm = df_lstm.copy(deep=True)
df_lstm_norm.Blobs = df_lstm_norm.Blobs.map(normalize_blobs)
lengths = []
for index, row in df_lstm_norm.iterrows():
    lengths.append(row.Blobs.shape[0])
df_lstm_norm["BlobCount"] = lengths
df_lstm_norm.head()

,userID,TaskID,VersionID,Blobs,BlobCount
6,17,16,10,"[[[0.0, 227.0, 0.0, 1.0, 0.0, 0.0, 4.0, 1.0, 1...",50
7,17,16,11,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0...",50
9,17,16,14,"[[[0.0, 0.0, 1.0, 0.0, 0.0, 2.0, 156.0, 1.0, 0...",50
10,17,16,18,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 2, 1...",50
11,17,16,19,"[[[2.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",50


In [137]:
df_lstm_norm.to_pickle("PklData/df_lstm_norm.pkl")